- h5py is a common package to interact with a dataset that is stored on an H5 file.
- matplotlib is a famous library to plot graphs in Python.
- PIL and scipy are used here to test your model with your own picture at the end.
- [numpy](https://www.numpy.org/) is the fundamental package for scientific computing with Python.
- [sklearn](http://scikit-learn.org/stable/) provides simple and efficient tools for data mining and data analysis. 
- [matplotlib](http://matplotlib.org) is a library for plotting graphs in Python.

In [31]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage

np.random.seed(1)

### Forward Activations

In [32]:
#activations: binary_step, linear_slope, sigmoid, tanh, relu, leaky_relu, parameterized_relu, elu, swish, softmax
def binary_step(z):
    if x<0: A = 0
    else: A = 1
    chache = z
    return A, cache
def linear_slope(z,slope):
    A = slope*z
    cache = z
    return A, cache
def sigmoid(z):
    A = 1/(1+ np.exp(-1*z))
    cache = z
    return A, cache
def tanh(z):
    A = np.tanh(z)
    cache = z
    return A, cache
def relu(z):
    A = max(0,z)
    cache = z
    return A, cache
def leaky_relu(z):
    if z>0: A=z
    else: A = 0.01*z
    cache = z
    return A, cache
def parameterized_relu(z,param):
    if z>0: A=z
    else: A = param*z
    cache=z
    return A, cache
def elu(z,param):
    if z<0: A=param*(np.exp(x)-1)
    else: A=z
    cache=z
    return A, cache
def swish(z):
    A = z/(1+np.exp(-1*z))
    cache = z
    return A, cache
def softmax(z):
    shiftx = x - np.max(x)
    exps = np.exp(shiftx)
    A = exps / np.sum(exps)
    cache=z
    return A, cache

### Backward Propogations for all activations 

In [40]:
#activations: binary_step, linear_slope, sigmoid, tanh, relu, leaky_relu, parameterized_relu, elu, swish, softmax
def binary_step_backward(dA, cache):
    Z = cache
    dZ = dA*0
    assert(dZ.shape==Z.shape)
    return dZ

def linear_slope_backward(dA, cache, slope):
    Z = cache
    dZ = dA*slope
    assert(dZ.shape==Z.shape)
    return dZ

def sigmoid_backward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    assert (dZ.shape == Z.shape)
    return dZ
def tanh_backward(dA, cache):
    Z = cache
    t = np.tanh(Z)
    dZ = 1- np.power(t,2)
    assert (dZ.shape == Z.shape)
    return dZ
def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    # When z <= 0, set dz to 0 
    dZ[Z <= 0] = 0
    assert (dZ.shape == Z.shape) 
    return dZ
def leaky_relu_backward(dA, cache):
    Z=cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0.01*dZ
    assert (dZ.shape == Z.shape) 
    return dZ
def parameterized_relu_backward(dA, cache, param):
    Z=cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = param*dZ
    assert (dZ.shape == Z.shape) 
    return dZ
def elu_backward(dA, cache, param):
    Z=cache
    dZ=a*np.exp(Z)
    assert (dZ.shape == Z.shape) 
    return dZ
def swish_backward(dA, cache):
    Z=cache
    sig = 1/(1+np.exp(-Z))
    swish = Z/(1+np.exp(-1*Z))
    dZ = swish + sig*(1-swish)
    assert (dZ.shape == Z.shape) 
    return dZ

#helper function to compute derivative of softmax function
def softmax_grad(softmax):
    # Reshape the 1-d softmax to 2-d so that np.dot will do the matrix multiplication
    s = softmax.reshape(-1,1)
    return np.diagflat(s) - np.dot(s, s.T)
#actual softmax function
def softmax_backward(dA, cache){
    Z = cache
    grad  = softmax_grad(Z)
    dZ = dA*Z    
}

In [44]:
#trash code
def softmax_backward(dA, cache):
    Z=cache
    dZ = np.copy(dA)
    for i in range(dZ.shape[0]):
        for j in range(dZ.shape[1]):
            if i == j:
                dZ[i,j] = dA[i][j] * (1-dA[i][j])
            else: 
                dZ[i,j] = -dA[i][j] * dA[j][i]
    assert (dZ.shape == Z.shape)
    return dZ
arr = np.array([[1.,2.,3.]])
print("shape:"+str(arr.shape))
print("type:"+str(type(arr)))
dZ=softmax_backward(arr,arr)
print(dZ)

shape:(1, 3)
type:<class 'numpy.ndarray'>


IndexError: index 1 is out of bounds for axis 0 with size 1

Initialize Parameters

In [9]:
def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):
        #(≈ 2 lines of code)
        # parameters['W' + str(l)] = ...
        # parameters['b' + str(l)] = ...
        # YOUR CODE STARTS HERE
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros([layer_dims[l],1])
        
        # YOUR CODE ENDS HERE
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

Linear Forward

In [10]:
def linear_forward(A, W, b):
    """
    Implements the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python tuple containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    Z = np.dot(W,A) + b
    cache = (A, W, b)
    
    return Z, cache

Linear Activation Forward

In [11]:
def linear_activation_forward(A_prev, W, b, activation):
    """

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python tuple containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """

    Z, linear_cache = linear_forward(A_prev,W,b)
    
    #activations: binary_step, linear, sigmoid, tanh, relu, leaky_relu, parameterized_relu, elu, swish, softmax
    if activation == "binary_step":
        A, activation_cache = binary
    
    elif activation == "linear_slope":
        A, activation_cache = linear_slope(Z)
    
    elif activation == "sigmoid":
        A, activation_cache = sigmoid(Z)
    
    elif activation == "tanh":
        A, activation_cache = tanh(Z)
    
    elif activation == "relu":        
        A, activation_cache = relu(Z)
        
    elif activation == "leaky_relu":        
        A, activation_cache = leaky_relu(Z)
        
    elif activation == "parameterized_relu":        
        A, activation_cache = parameterized_relu(Z)
        
    elif activation == "elu":        
        A, activation_cache = elu(Z)
                
    elif activation == "swish":        
        A, activation_cache = swish(Z)

    elif activation == "softmax":        
        A, activation_cache = softmax(Z)
        
        
    #collect cache
    cache = (linear_cache, activation_cache)

    return A, cache

L Model Forward

In [ ]:
def L_model_forward(X, parameters,layers):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    layers -- a list of layers that need to be applied to the network
    
    Returns:
    AL -- activation value from the output (last) layer
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L of them, indexed from 0 to L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2 # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    # The for loop starts at 1 because layer 0 is the input
    for l in range(1, L):
        A_prev = A 
        #(≈ 2 lines of code)
        # A, cache = ...
        # caches ...
        # YOUR CODE STARTS HERE
        A,cache = linear_activation_forward(A_prev,parameters['W'+str(l)],parameters['b'+str(l)],layers[l-1])
        caches.append(cache)
        # YOUR CODE ENDS HERE
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    #(≈ 2 lines of code)
    # AL, cache = ...
    # caches ...
    # YOUR CODE STARTS HERE
    AL, cache = linear_activation_forward(A,parameters['W'+str(L)],parameters['b'+str(L)],layers[L-1])
    caches.append(cache)
    # YOUR CODE ENDS HERE
          
    return AL, caches

Cost Function

In [ ]:
def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    # (≈ 1 lines of code)
    # cost = ...
    # YOUR CODE STARTS HERE
    cost = (-1/m)* (np.dot(Y,np.log(AL.T)) + np.dot(1-Y,np.log(1-AL.T)))
    
    # YOUR CODE ENDS HERE
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).

    
    return cost

Linear Backward

In [ ]:
def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    # dW = ...
    # db = ... sum by the rows of dZ with keepdims=True
    # dA_prev = ...
    # YOUR CODE STARTS HERE
    dW = (1/m)*np.dot(dZ,A_prev.T)
    db = (1/m)*np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    # YOUR CODE ENDS HERE
    
    return dA_prev, dW, db

Linear Activation Backward

In [ ]:
def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    #retrieve cache
    linear_cache, activation_cache = cache
    
    #find dZ based on layer's activation
    if activation == "binary_step":
        dZ = binary_step_backward(dA,activation_cache)
    
    elif activation == "linear":
        dZ = linear_slope_backward(dA,activation_cache)
    
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA,activation_cache)
    
    elif activation == "tanh":
        dZ = tanh_backward(dA,activation_cache)
    
    elif activation == "relu":        
        dZ = relu_backward(dA,activation_cache)
        
    elif activation == "leaky_relu":        
        dZ = leaky_relu_backward(dA,activation_cache)
        
    elif activation == "parameterized_relu":        
        dZ = parameterized_relu_backward(dA,activation_cache)
        
    elif activation == "elu":        
        dZ = elu_backward(dA,activation_cache)
                
    elif activation == "swish":        
        dZ = swish_backward(dA,activation_cache)

    elif activation == "softmax":        
        dZ = softmax_backward(dA,activation_cache)
    
    #use dZ to find dA_prev, dW, db
    dA_prev, dW, db = linear_backward(dZ,linear_cache)
    
    return dA_prev, dW, db

L Model Backward

In [ ]:
def L_model_backward(AL, Y, caches, layers):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    dAL = -(np.divide(Y,AL) - np.divide(1-Y,1-AL))
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    #(approx. 5 lines)
    # current_cache = ...
    # dA_prev_temp, dW_temp, db_temp = ...
    # grads["dA" + str(L-1)] = ...
    # grads["dW" + str(L)] = ...
    # grads["db" + str(L)] = ...
    # YOUR CODE STARTS HERE
    current_cache = caches[L-1]
    dA_prev_temp,dw_temp,db_temp = linear_activation_backward(dAL,current_cache,layers[L-1])
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dw_temp
    grads["db" + str(L)] = db_temp
    
    # YOUR CODE ENDS HERE
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)]
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" +str(l+1)],current_cache,layers[l])
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l+1)] = dW_temp
        grads["db" + str(l+1)] = db_temp
        
    return grads

Update Parameters

In [ ]:
def update_parameters(params, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    params -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    parameters = params.copy()
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    #(≈ 2 lines of code)
    for l in range(L):
        # parameters["W" + str(l+1)] = ...
        # parameters["b" + str(l+1)] = ...
        # YOUR CODE STARTS HERE
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*grads["dW"+str(l+1)] 
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*grads["db"+str(l+1)]
        
        # YOUR CODE ENDS HERE
    return parameters

L Layer Model

In [ ]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- data, numpy array of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization.
    #(≈ 1 line of code)
    # parameters = ...
    # YOUR CODE STARTS HERE
    parameters = initialize_parameters_deep(layers_dims)
    
    # YOUR CODE ENDS HERE
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        #(≈ 1 line of code)
        # AL, caches = ...
        # YOUR CODE STARTS HERE
        AL, caches = L_model_forward(X,parameters)
        
        # YOUR CODE ENDS HERE
        
        # Compute cost.
        #(≈ 1 line of code)
        # cost = ...
        # YOUR CODE STARTS HERE
        cost = compute_cost(AL,Y)
        
        # YOUR CODE ENDS HERE
    
        # Backward propagation.
        #(≈ 1 line of code)
        # grads = ...    
        # YOUR CODE STARTS HERE
        grads = L_model_backward(AL,Y,caches)
        
        # YOUR CODE ENDS HERE
 
        # Update parameters.
        #(≈ 1 line of code)
        # parameters = ...
        # YOUR CODE STARTS HERE
        parameters = update_parameters(parameters,grads,learning_rate)
        
        # YOUR CODE ENDS HERE
                
        # Print the cost every 100 iterations
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    
    return parameters, costs

In [ ]:
parameters, costs = L_layer_model(train_x, train_y, layers_dims, num_iterations = 2500, print_cost = True)

In [ ]:
X, Y = load_planar_dataset()

In [ ]:
# Visualize the data:
plt.scatter(X[0, :], X[1, :], c=Y, s=40, cmap=plt.cm.Spectral);

In [ ]:
# Train the logistic regression classifier
clf = sklearn.linear_model.LogisticRegressionCV();
clf.fit(X.T, Y.T);